# Own your health app!

Trying to show a simple PoC where I give it a medical test document and then ask 
some structured responses from it. 

For our real app, we will have 10's or 100's of such documents per individual 
and each document could be 10's to 100's of pages. This is because someone's diagnostic 
journey e.g. cancer is spread across tests and visits to 100's of institutions before they
get refered to a large cancer center oncologist and this oncology team has to make sense of all you have
endured {treatment, outcomes, discharge summaries} to give you the right next treatment when 
you arrive at their doorstep.


For now success will be if for a single report I am able to get back all the
diagnostic tests done without missing. It tends to miss a few and on repeated nudging since 
I know the answer, pull out more and more tests missed previously.
Also, as a bonus what I would really really like would be I ask a question with a schema 
and it returns *all* the elements in the same schema.


In [238]:
from langchain.document_loaders import UnstructuredPDFLoader

# Right now this does not do directory, it takes one file at a time, could this be a limitation later !?!?

## First approach is doing this with one big gulp, no splitting and using function calls to structure

In [267]:
## Core algorithm which parses the PDF file and structres the output.
# the plus of using pedantic models is yo can force what you want the return obect to look like
# pedantic then does the validation to make sure!
# In the future we can make this configurable!

#TODO: Adding a list object into the pedayntic object which itself is an object?


from pydantic import BaseModel, Field
from langchain.chains.openai_functions.extraction import _get_extraction_function
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

doc = UnstructuredPDFLoader(file_path="/Users/vinayak/projects/medical_records_parser/data/MinnieMouseReport.pdf")
# Above I read the whole thing as ONE large blob, this was possible since the file is only 7 pages!
# if the file becomes too large, this is not possible.
docs = doc.load()


query = """
You are an expert medical transcriber. Please give me back a table of all the analytes measured. The table should have following columns: analyte_measured, result, reference_interval, unit, notes. 
If a patricular column does not exist please say NA.
Please double check your work and do not miss any analyte.
"""


class Analyte(BaseModel):
    """Information about an analyte."""
    analyte_measured: str
    result: str
    reference_interval: str
    unit: str
    notes: str

openai_function = _get_extraction_function(Analyte.schema())

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo-16k")

prompt = ChatPromptTemplate.from_messages([
    ("system", query),
    ("user", "{doc}")
])


output_parser = JsonKeyOutputFunctionsParser(key_name="info")
chain = prompt | model.bind(functions=[openai_function], function_call={"name": "information_extraction"}) | output_parser

response = chain.invoke({"doc": docs[0].page_content})

In [269]:
pd.DataFrame(response) # Amit/Guy can you please see if the output is correct?

,analyte_measured,result,reference_interval,unit,notes
0,BLOOD UREA NITROGEN,7,7.0 - 17.0,mg/dL,METHOD : UREASE WITH INDICATOR DYE
1,"CREATININE, SERUM",0.62,0.52 - 1.04,mg/dL,METHOD : ENZYMETIC IDMS
2,BUN/CREAT RATIO,11.29,NA,NA,NA
3,"URIC ACID, SERUM",4.5,2.5 - 6.2,mg/dL,METHOD : URICASE UV
4,"TOTAL PROTEIN, SERUM",6.4,6.3 - 8.30,g/dL,"METHOD : BIURET, END POINT"
5,"ALBUMIN, SERUM",2.9,3.5 - 5.0,g/dL,METHOD : BCG DYE BINDING METHOD
6,GLOBULIN,3.5,2.0 - 3.5,g/dL,METHOD : CALCULATED PARAMETER
7,SODIUM,139,137 - 145,mmol/L,METHOD : ION SELECTIVE ELECTRODE TECHNOLOGY
8,POTASSIUM,3.40,3.6 - 5.0,mmol/L,METHOD : ION SELECTIVE ELECTRODE TECHNOLOGY
9,CHLORIDE,107,98 - 107,mmol/L,METHOD : ION SELECTIVE ELECTRODE TECHNOLOGY


## Second approach is doing the same thing as above but one page at a time since sometimes the file might be too large to fit context size of the model.

In [270]:
from langchain.document_loaders import UnstructuredPDFLoader
doc = UnstructuredPDFLoader(file_path="/Users/vinayak/projects/medical_records_parser/data/MinnieMouseReport.pdf", mode="paged")
# Above is the key difference where it is loading the data as "paged" mode.

docs = doc.load()

query = """
Please give me back a table of all the analytes measured. The table should have following columns: analyte_measured, result, reference_interval, unit, notes. 
If a patricular column does not exist please say NA.
Please double check your work and do not miss any analyte.
"""

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

model = ChatOpenAI(model="gpt-3.5-turbo-16k")

prompt = ChatPromptTemplate.from_messages([
    ("system", query),
    ("user", "{doc}")
])

# This is because of what _get_extraction_function does
output_parser = JsonKeyOutputFunctionsParser(key_name="info")

chain = prompt | model.bind(functions=[openai_function], function_call={"name": "information_extraction"}) | output_parser


responses = chain.batch([{"doc": d.page_content} for d in docs], {"max_concurrency": 5})

extracted_by_function_call = []
for response in responses:
    extracted_by_function_call.extend(response)
    


In [271]:
pd.DataFrame(extracted_by_function_call)

,analyte_measured,result,reference_interval,unit,notes
0,SERUM BLOOD UREA NITROGEN,7,7.0 - 17.0,mg/dL,METHOD : UREASE WITH INDICATOR DYE
1,"CREATININE, SERUM",0.62,0.52 - 1.04,mg/dL,METHOD : ENZYMETIC IDMS
2,BUN/CREAT RATIO,11.29,,,
3,"URIC ACID, SERUM",4.5,2.5 - 6.2,mg/dL,METHOD : URICASE UV
4,"TOTAL PROTEIN, SERUM",6.4,6.3 - 8.30,g/dL,"METHOD : BIURET, END POINT"
...,...,...,...,...,...
63,ASPARTATE AMINOTRANSFERASE (AST/SGOT),35,14 - 36,U/L,NA
64,ALANINE AMINOTRANSFERASE (ALT/SGPT),27,< 35.0,U/L,NA
65,ALKALINE PHOSPHATASE,87,38 - 126,U/L,NA
66,GAMMA GLUTAMYL TRANSFERASE (GGT),25,12 - 43,U/L,NA


## Now for the third approach, this is a VERY large document of 130+ pages including mishmash of diffrent kind of reports since it has come from an EMR dump (likely EPIC)

In [36]:
doc = UnstructuredPDFLoader(file_path="/Users/vinayak/projects/kaiser/data/Barbara/UCLA Health.pdf")
docs = doc.load()

In [48]:
#Showing the entire contents of the document
docs[0].page_content

'SM ORTHOPAEDICS 1225 15TH ST # 2100 SANTA MONICA CA 90404-1101\n\nLewy, Barbara E MRN: 2107701, DOB: 9/11/1946, Sex: F Visit date: 5/26/2023\n\n05/26/2023 - Office Visit in UCLA Orthopaedic Surgery\n\nProgress Notes\n\nProgress Notes by Stancil, Ryan D., MD, MPH at 5/26/2023 1300\n\nAuthor: Stancil, Ryan D., MD, MPH Filed: 06/05/23 1047 Status: Signed\n\nService: — Encounter Date: 5/26/2023 Editor: Stancil, Ryan D., MD, MPH (Physician)\n\nAuthor Type: Physician Creation Time: 05/31/23 1606\n\nUCLA Department of Orthopaedic Surgery New patient\n\nPatient: Barbara E Lewy MRN: 2107701 DOB: 9/11/1946 Date of Visit: 5/26/2023 Referred By: Referral, Self Primary Care Physician: Biserka Matkovic, MD\n\nChief Complaint: left Knee Pain\n\nHistory of Present Illness\n\nBarbara E Lewy is a 76 y.o. female with a history of chronic left knee pain, with no inciting event, but which has worsened over the past several months. The pain is localized to medial knee, and is worse with activity, especiall

In [106]:
# we need some kind of splitter which closely resembles where records start and end
# I am using the default on, which is sub optimal and does a lot of repeats and not so useful summarization too!

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 10000,
    chunk_overlap  = 500,
    length_function = len,
    is_separator_regex = False,
)

doc = UnstructuredPDFLoader("/Users/vinayak/projects/kaiser/data/Barbara/UCLA Health.pdf")

docs = doc.load_and_split(text_splitter)

print("Number of splits %d"%(len(docs)))


### Notice I also changed the question from parsing a diagnostic report to parsing medical visits. Likely we will have to do a hybrid where we first split the very large document into different pieces and for each piece parse what is relevant (diagnostic report vs visits vs pathology report)

In [111]:
query = """
You are an expert medical transcriber and can transcribe electronic health records with great skill.
Please give me back a table of all the visits from the patient. Columns to return are:
patient_name, date_of_visit, category, provider, institution, brief_summary
The category can only be one of the following values: LAB_REPORT, PATHOLOGY, RADIOLOGY, PROCEDURE, DIAGNOSTIC_TEST, ROUTINE_VISIT
If a patricular column does not exist please say NA.
Please double check your work and do not miss any visits.
"""


from pydantic import BaseModel, Field
from langchain.chains.openai_functions.extraction import _get_extraction_function

class Visit(BaseModel):
    """Information about visit to the medical facility."""
    patient_name: str
    date_of_visit: str
    category: str
    provider: str
    institution: str
    brief_summary: str

openai_function = _get_extraction_function(Visit.schema())

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo-16k")

prompt = ChatPromptTemplate.from_messages([
    ("system", query),
    ("user", "{doc}")
])

chain = prompt | model.bind(functions=[openai_function], function_call={"name": "information_extraction"}) | output_parser

# Make subset of docs below (8) so I don't become bankrupt! with openAI bills

responses = chain.batch([{"doc": d.page_content} for d in docs], {"max_concurrency": 5})


## The above code aks a question per subset of the data (according to the split which is 10k). This means it will have
## answers per split. The response object is list of response, each response hiving a list of dicts

In [178]:
# we need to flatten the list

flattened_list = list()
for d in responses:
    flattened_list.extend(d)

flattened_list

[{'patient_name': 'Barbara E Lewy',
  'date_of_visit': '5/26/2023',
  'category': 'ROUTINE_VISIT',
  'provider': 'Stancil, Ryan D., MD, MPH',
  'institution': 'UCLA Orthopaedic Surgery',
  'brief_summary': 'Chief Complaint: left Knee Pain'},
 {'patient_name': 'Barbara E Lewy',
  'date_of_visit': '5/26/2023',
  'category': 'ROUTINE_VISIT',
  'provider': 'UCLA Orthopaedic Surgery',
  'institution': 'SM ORTHOPAEDICS',
  'brief_summary': 'Office Visit'},
 {'patient_name': 'Barbara E Lewy',
  'date_of_visit': '5/26/2023',
  'category': 'ROUTINE_VISIT',
  'provider': 'UCLA Orthopaedic Surgery',
  'institution': 'SM ORTHOPAEDICS',
  'brief_summary': 'Office Visit in UCLA Orthopaedic Surgery'},
 {'patient_name': 'Barbara E Lewy',
  'date_of_visit': '5/26/2023',
  'category': 'ROUTINE_VISIT',
  'provider': 'Adam A. Sassoon',
  'institution': 'UCLA Orthopaedic Surgery',
  'brief_summary': 'Barbara E Lewy visited UCLA Orthopaedic Surgery for a routine visit. No acute abnormalities were found in t

In [180]:
import pandas as pd
df = pd.DataFrame.from_records(flattened_list)
df


In [231]:
#Doing cleanup to remove junk, this is because our parser is not yet good enough.

df1 = df[(df.patient_name != 'NA')]
df1 = df1[(df1.category != 'NA')]

df1['date_cleanedup']= pd.to_datetime(df1['date_of_visit'], format='mixed')
df1['final_date'] = df1['date_cleanedup'].apply(lambda x: x.strftime('%B %d, %Y'))

df1.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)

df1.sort_values(by='date_cleanedup').to_csv('/Users/vinayak/projects/df_to_test.tsv', sep="\t", index=False)




In [237]:
df1.rename(columns={'final_date': 'title','institution': 'cardTitle', 'category': 'cardSubtitle', 'brief_summary': 'cardDetailedText' }, inplace=True)
cols_needed = ['title', 'cardTitle', 'cardSubtitle', 'cardDetailedText']
df1[cols_needed].to_dict('records')

[{'title': 'May 26, 2023',
  'cardTitle': 'UCLA Orthopaedic Surgery',
  'cardSubtitle': 'ROUTINE_VISIT',
  'cardDetailedText': 'Chief Complaint: left Knee Pain'},
 {'title': 'May 26, 2023',
  'cardTitle': 'SM ORTHOPAEDICS',
  'cardSubtitle': 'ROUTINE_VISIT',
  'cardDetailedText': 'Office Visit'},
 {'title': 'May 26, 2023',
  'cardTitle': 'SM ORTHOPAEDICS',
  'cardSubtitle': 'ROUTINE_VISIT',
  'cardDetailedText': 'Office Visit in UCLA Orthopaedic Surgery'},
 {'title': 'May 26, 2023',
  'cardTitle': 'UCLA Orthopaedic Surgery',
  'cardSubtitle': 'ROUTINE_VISIT',
  'cardDetailedText': 'Barbara E Lewy visited UCLA Orthopaedic Surgery for a routine visit. No acute abnormalities were found in the examination. X-ray of the bilateral knees was reviewed and showed no interval complications. Treatment plan includes VMO strengthening exercises and avoiding impact loading exercises.'},
 {'title': 'May 26, 2023',
  'cardTitle': 'UCLA Orthopaedic Surgery',
  'cardSubtitle': 'ROUTINE_VISIT',
  'cardDe

# Scratch space

### The stuff below is working but chunking is per page

In [117]:
from langchain.document_loaders import UnstructuredPDFLoader
doc = UnstructuredPDFLoader(file_path="/Users/vinayak/projects/kaiser/data/Barbara/UCLA Health.pdf", mode="paged")

docs = doc.load()

query = """
Please give me back a table of all the visits from the patient. Columns to return are:
visit_date, visit_reason, visit_department, visit_summary
If a patricular column does not exist please say NA.
Please double check your work and do not miss any visits.
"""

class Visit(BaseModel):
    """Information about visit to the medical facility."""
    visit_date: str
    visit_reason: str
    visit_department: str
    visit_summary: str



openai_function = _get_extraction_function(Visit.schema())

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

model = ChatOpenAI(model="gpt-3.5-turbo-16k")

prompt = ChatPromptTemplate.from_messages([
    ("system", query),
    ("user", "{doc}")
])

# This is because of what _get_extraction_function does
output_parser = JsonKeyOutputFunctionsParser(key_name="info")

chain = prompt | model.bind(functions=[openai_function], function_call={"name": "information_extraction"}) | output_parser


responses = chain.batch([{"doc": d.page_content} for d in docs], {"max_concurrency": 5})

In [50]:
responses

[[{'visit_date': '5/26/2023',
   'visit_reason': 'Office Visit in UCLA Orthopaedic Surgery',
   'visit_department': 'UCLA Orthopaedic Surgery',
   'visit_summary': 'Barbara E Lewy is a 76 y.o. female with a history of chronic left knee pain, with no inciting event, but which has worsened over the past several months. The pain is localized to medial knee, and is worse with activity, especially ascending and descending stairs. Patient denies radiation of pain and associated paresthesias, including numbness and tingling. To date, the patient has attempted activity modification, heat, ice, physical therapy with home exercises, NSAIDs without significant relief. Pain is primarily with side to side activity or with heavy stair use. History of L TKA in 2005, R TKA in 2018. No complaints with R knee. Overall she is functional to her desires and the L knee is not holding her back from desired activities'}],
 [{'visit_date': '5/26/2023',
   'visit_reason': 'Office Visit',
   'visit_department': 